# Laboratorium 5 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas scipy matplotlib`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds


from random import choice, sample
from statistics import mean, stdev

In [2]:
PATH = 'ml-latest-small'

In [19]:
raw_ratings = pd.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
movies = list(raw_ratings['movieId'].unique())
users = list(raw_ratings['userId'].unique())

In [3]:
# wczytujemy oceny uzytkownikow i obliczamy (za pomoc dekompozycji macierzy) wszystkie przewidywane oceny filmow

def read_ratings(path, k=600, scale_factor=2.0, print_stats=True):
    # idea: https://www.kaggle.com/code/indralin/movielens-project-1-2-collaborative-filtering
    reviews = pd.read_csv(f'{path}/ratings.csv', names=['userId', 'movieId', 'rating', 'time'], delimiter=',', engine='python', skiprows=1)

    reviews.drop(['time'], axis=1, inplace=True)
    reviews_no, _ = reviews.shape
    reviews_matrix = reviews.pivot(index='userId', columns='movieId', values='rating')
    movies = reviews_matrix.columns
    users = reviews_matrix.index
    users_no, movies_no = reviews_matrix.shape
    print(f'Got {reviews_no} reviews for {movies_no} movies and {users_no} users.')

    user_ratings_mean = np.nanmean(reviews_matrix.values, axis=1)
    normalized_reviews_matrix = np.nan_to_num(reviews_matrix.values - user_ratings_mean.reshape(-1, 1), 0.0)

    U, sigma, Vt = svds(normalized_reviews_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1).clip(0.5, 5.0)
    mean_square_error = np.nanmean(np.square(predicted_ratings - reviews_matrix.values))
    std_square_error = np.nanstd(np.square(predicted_ratings - reviews_matrix.values))
    print(f'Reviews prediction mean square error = {mean_square_error}')
    print(f'Reviews prediction standatd deviation of square error = {std_square_error}')

    if print_stats:
        stats = [
            ('metric', 'dataset', 'prediction'),
            ('avg', np.nanmean(reviews_matrix), np.mean(predicted_ratings)),
            ('st_dev', np.nanstd(reviews_matrix), np.std(predicted_ratings)),
            ('median', np.nanmedian(reviews_matrix), np.median(predicted_ratings)),
            ('p25', np.nanquantile(reviews_matrix, 0.25), np.quantile(predicted_ratings, 0.25)),
            ('p75', np.nanquantile(reviews_matrix, 0.75), np.quantile(predicted_ratings, 0.75))
        ]
        print('Stats (for raings in original range [0.5, 5.0]):')
        print('\n'.join([str(s) for s in stats]))

    rounded_predictions = np.rint(scale_factor * predicted_ratings) # cast values to {1, 2, ..., 10}
    return pd.DataFrame(data=rounded_predictions, index=list(users), columns=list(movies))

ratings = read_ratings(PATH)
# dostep do danych:
# ratings[movieId][userId] pobiera 1 wartosc
# ratings.loc[:, movieId] pobiera wektor dla danego filmu
# ratings.loc[userId, :] pobiera wektor dla danego uzytkownika
ratings

Got 100836 reviews for 9724 movies and 610 users.
Reviews prediction mean square error = 1.6577787842924556e-05
Reviews prediction standatd deviation of square error = 0.0007928950536518169
Stats (for raings in original range [0.5, 5.0]):
('metric', 'dataset', 'prediction')
('avg', 3.501556983616962, 3.657222337747399)
('st_dev', 1.042524069618056, 0.49546237560971024)
('median', 3.5, 3.7052240088117694)
('p25', 3.0, 3.357451718316402)
('p75', 4.0, 3.999981626883001)


,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,8.0,9.0,8.0,9.0,9.0,8.0,9.0,9.0,9.0,9.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
5,8.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,5.0,7.0,7.0,7.0,7.0,7.0,5.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
607,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
608,5.0,4.0,4.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
609,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [4]:
# wczytujemy nazwy filmow i kategorie

movies_metadata = pd.read_csv('ml-latest-small/movies.csv').set_index('movieId')
movies_metadata

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [5]:
# wczytujemy przykladowe grupy uzytkownikow
groups = pd.read_csv('groups.csv').values.tolist()
groups

[[111, 307, 474, 599, 414],
 [469, 182, 232, 448, 600],
 [508, 581, 497, 402, 566],
 [300, 515, 245, 568, 507],
 [2, 371, 252, 518, 37],
 [269, 360, 469, 287, 308],
 [243, 527, 418, 118, 370],
 [186, 559, 327, 553, 314]]

In [6]:
# przygotowujemy funkcje pomocnicza

def describe_group(group, N=10):
    print(f'\n\nUser ids: {group}')
    group_size = len(group)

    mean_stdev = ratings.loc[group].std(axis=0).mean()
    median_stdev = ratings.loc[group].std(axis=0).median()
    std_stdev = ratings.loc[group].std(axis=0).std()
    print(f'\nMean ratings deviation: {mean_stdev}')
    print(f'Median ratings deviation: {median_stdev}')
    print(f'Standard deviation of ratings deviation: {std_stdev}')

    average_scores = ratings.iloc[group].mean(axis=0)
    average_scores = average_scores.sort_values()
    best_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[-N:].index)]
    worst_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[:N].index)]

    print('\nBest movies:')
    for movie, score in best_movies[::-1]:
        print(f'{movie}, {score}*')
    print('\nWorst movies:')
    for movie, score in worst_movies:
        print(f'{movie}, {score}*')

describe_group(groups[5])



User ids: [269, 360, 469, 287, 308]

Mean ratings deviation: 1.1259149574579788
Median ratings deviation: 1.0954451150103321
Standard deviation of ratings deviation: 0.17836724055771178

Best movies:
Toy Story (1995), 8.2*
Forrest Gump (1994), 8.2*
Willy Wonka & the Chocolate Factory (1971), 8.0*
Braveheart (1995), 8.0*
Terminator 2: Judgment Day (1991), 7.8*
Schindler's List (1993), 7.8*
Dances with Wolves (1990), 7.6*
James and the Giant Peach (1996), 7.6*
Dead Man Walking (1995), 7.6*
Nixon (1995), 7.6*

Worst movies:
Broken Arrow (1996), 5.2*
Sleepy Hollow (1999), 5.4*
The Devil's Advocate (1997), 5.4*
Cable Guy, The (1996), 5.4*
Mission: Impossible (1996), 5.6*
Nutty Professor, The (1996), 5.6*
Down Periscope (1996), 5.8*
Cheech and Chong's Up in Smoke (1978), 5.8*
Wrong Man, The (1956), 5.8*
Fog, The (2005), 5.8*


## Część 2. - algorytmy proste

In [7]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania

class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'

    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [8]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'

    def get_avg_score(self, movie_id, ratings, group):
        return sum(ratings[movie_id][user] for user in group) / len(group)

    def recommend(self, movies, ratings, group, size):
        return sorted(movies, key=lambda movie: self.get_avg_score(movie, ratings, group), reverse=True)[:size]

In [9]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold

    def get_avg_score(self, movie_id, ratings, group):
        return sum(ratings[movie_id][user] for user in group) / len(group)

    def min_movie_rating_in_group(self, movie_id, ratings, group):
        return min([ratings[movie_id][user] for user in group])

    def recommend(self, movies, ratings, group, size):
        movies_without_misery = [m for m in movies if self.min_movie_rating_in_group(m, ratings, group) >= self.score_threshold]
        return sorted(movies, key=lambda movie: self.get_avg_score(movie, ratings, group), reverse=True)[:size]

In [10]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika w kazdej iteracji

class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'

    def recommend(self, movies, ratings, group, size):
        recommendations = []
        for i in range(size):
            ith_user_rating_movies_sorted = sorted(movies, key = lambda movie: ratings[movie][group[i%len(group)]], reverse=True)
            for movie in ith_user_rating_movies_sorted:
                if movie not in recommendations:
                    recommendations.append(movie)
                    break
        return recommendations

In [13]:
# wybrany algorytm wyborczy (dyktatura, Borda, Copeland)

class VotingRecommender(Recommender):
    def __init__(self):
        self.name = 'voting'

    def recommend(self, movies, ratings, group, size):
        # worst user movie gets 0, second worst 1, etc.
        movies_scores = {movie: 0 for movie in movies}
        for user_id in group:
            for i, movie_id in enumerate(ratings.loc[user_id].sort_values(ascending=False).index):
                movies_scores[movie_id] += i
        return list(sorted(movies_scores, key=movies_scores.get, reverse=True)[:size])

In [12]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class ProportionalApprovalVotingRecommender(Recommender):
    def __init__(self, threshold):
        self.threshold = threshold
        self.name = 'PAV'

    def count_score(self, ratings, movie_id, group, group_movies_counter):
        score = 0
        for user in group:
            if ratings[movie_id][user] >= self.threshold:
                score += 1 / (group_movies_counter[user] + 1)
        return score

    def recommend(self, movies, ratings, group, size):
        from collections import defaultdict
        group_movies_counter = defaultdict(int)
        recommendations = []
        movie_set = set(movies)
        for i in range(size):
            best_movie = sorted(list(movie_set), key=lambda movie: self.count_score(ratings, movie, group, group_movies_counter),reverse=True)[0]
            movie_set.remove(best_movie)
            for user in group:
                if ratings[best_movie][user] >= self.threshold:
                    group_movies_counter[user] += 1
            recommendations.append(best_movie)
        return recommendations

## Część 3. - funkcje celu

In [14]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

def top_n_movies_for_user(ratings, movies, user_id, n):
    movies = []
    for movie_id in ratings.loc[user_id].sort_values(ascending=False).index:
        if len(movies) >= n:
            break
        movies.append(movie_id)
    return movies

def total_score(recommendation, user_id, ratings):
    return sum([ratings.loc[user_id][movie_id] for movie_id in recommendation])

In [15]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej
def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    return total_score(recommendation, user_id, ratings) / total_score(top_n_movies_for_user(ratings, movies, user_id, len(recommendation)), user_id, ratings)

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie
def overall_group_satisfaction(recommendation, group, movies, ratings):
    return mean([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group])

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_disagreement(recommendation, group, movies, ratings):
    return max([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group]) - min([overall_user_satisfaction(recommendation, user_id, movies, ratings) for user_id in group])

## Część 4. - Sequential Hybrid Aggregation

In [16]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie
class SequentialHybridAggregationRecommender(Recommender):
    def __init__(self):
        self.name = 'sequential_hybrid_aggregation'

    def recommend(self, movies, ratings, group, size):
        # Calculate average score and least score for each movie in the group
        avg_score = ratings.loc[group].mean(axis=0)
        least_score = ratings.loc[group].min()
        alpha = 1

        # Create a dictionary to store scores for each movie
        score = {movie: 0 for movie in ratings.columns}

        recommendation = []

        # Iterate through the specified number of recommendations
        for _ in range(size):
            # Calculate the score for each movie using the weighted average
            score.update(
                {movie: (1 - alpha) * avg_score.loc[movie] + alpha * least_score.loc[movie] for movie in movies})

            # Set the score to -1 for movies already recommended
            score.update({movie: -1 for movie in recommendation})

            # Find the movie with the maximum score
            max_score_movie = max(score.items(), key=lambda k: k[1])[0]
            recommendation.append(max_score_movie)

            # Update alpha using the group disagreement function
            alpha = group_disagreement(recommendation, group, movies, ratings)

        return recommendation

## Część 5. - porównanie algorytmów

In [20]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(5),
    FairnessRecommender(),
    VotingRecommender(),
    ProportionalApprovalVotingRecommender(5),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci obu funkcji celu dla kazdej rekomendacji
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    print(f'\n\n{recommender.name}')
    overall_group_satisfactions = []
    group_disagreements = []
    for group in groups:
        recommendation = recommender.recommend(movies, ratings, group, recommendation_size)
        overall_group_satisfaction_value = overall_group_satisfaction(recommendation, group, movies, ratings)
        group_disagreement_value = group_disagreement(recommendation, group, movies, ratings)
        overall_group_satisfactions.append(overall_group_satisfaction_value)
        group_disagreements.append(group_disagreement_value)
        print(f'\nRecommendation: {recommendation}')
        print(f'Overall group satisfaction: {overall_group_satisfaction_value}')
        print(f'Group disagreement: {group_disagreement_value}')
    print(f'\nMean overall group satisfaction: {mean(overall_group_satisfactions)}')
    print(f'Standard deviation of overall group satisfaction: {stdev(overall_group_satisfactions)}')
    print(f'Mean group disagreement: {mean(group_disagreements)}')
    print(f'Standard deviation of group disagreement: {stdev(group_disagreements)}')



random

Recommendation: [1357, 44773, 60950, 1334, 49263, 2027, 36519, 678, 2629, 6093]
Overall group satisfaction: 0.62
Group disagreement: 0.22999999999999998

Recommendation: [4526, 4116, 4735, 56251, 2207, 2671, 86593, 2291, 46948, 5177]
Overall group satisfaction: 0.642
Group disagreement: 0.13

Recommendation: [3784, 1824, 6539, 78105, 1556, 1190, 4446, 54617, 32234, 8773]
Overall group satisfaction: 0.7549003436426117
Group disagreement: 0.26116838487972516

Recommendation: [152284, 2376, 55687, 8934, 4942, 106489, 85774, 47646, 26357, 155774]
Overall group satisfaction: 0.8697804576376005
Group disagreement: 0.24242424242424243

Recommendation: [3851, 4323, 86066, 137337, 140541, 6962, 94503, 7022, 102066, 7292]
Overall group satisfaction: 0.8287528344671202
Group disagreement: 0.12222222222222223

Recommendation: [8454, 6810, 118985, 69644, 3790, 7169, 2520, 2377, 108727, 4366]
Overall group satisfaction: 0.6384126984126984
Group disagreement: 0.2777777777777778

Recommendat